This Python Jupyter Notebook creates the SQL Server tables for the UK General Election model, in accordance with the entity-relationship diagram shown below.

In [ ]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [ ]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [ ]:
CreateRegionTypes = """Create Table RegionTypes
    (RegionType VARCHAR(25) PRIMARY KEY NOT NULL)"""

In [ ]:
engine.execute(CreateRegionTypes)

In [ ]:
CreatePollsters = """Create Table Pollsters
    (
    PollsterName VARCHAR(100) PRIMARY KEY NOT NULL,
    DefaultRegionType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType)
    )"""

In [ ]:
engine.execute(CreatePollsters)

In [ ]:
CreateRegions = """Create Table Regions
    (
    RegionID AS RegionType + RegionName PERSISTED PRIMARY KEY NOT NULL,
    RegionName VARCHAR(50) NOT NULL UNIQUE,
    RegionType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType) NOT NULL	
    )"""

In [ ]:
engine.execute(CreateRegions)

In [ ]:
CreateParties = """Create Table Parties
    (
    PartyAbbreviation VARCHAR(7) PRIMARY KEY NOT NULL,
    PartyFullName VARCHAR(25)
    )"""

In [ ]:
engine.execute(CreateParties)

In [ ]:
CreateConstituencies = """Create Table Constituencies
    (
    ONSID CHAR(9) NOT NULL,
    ConstituencyName VARCHAR(50) PRIMARY KEY NOT NULL,
    County VARCHAR(25) NOT NULL,
    Nation VARCHAR(20) NOT NULL,
    ConstituencyType VARCHAR(10) NOT NULL,
    ITL1Region VARCHAR(25) NOT NULL,
    Latitude DECIMAL(10,8),
    Longitude DECIMAL(10,8),
    FirstParty VARCHAR(7) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    SecondParty VARCHAR(7) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    PreviousFirstParty VARCHAR(7) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    Electorate INT NOT NULL,
    ValidVotes INT NOT NULL,
    MajorityVotes INT NOT NULL,
    MajorityShare DECIMAL(9,8) NOT NULL,
    MPFirstName VARCHAR(25),
    MPSurname VARCHAR(25),
    MPGender VARCHAR(6),
    DeclarationTime datetime
    )"""

In [ ]:
engine.execute(CreateConstituencies)

In [ ]:
CreateRegionConstituencies = """Create Table RegionConstituencies
    (
    RegionConsID AS RegionID + ConstituencyName PERSISTED PRIMARY KEY NOT NULL,
    ConstituencyName VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName) NOT NULL,
    RegionID VARCHAR(75) FOREIGN KEY REFERENCES Regions(RegionID) NOT NULL 
    )"""

In [ ]:
engine.execute(CreateRegionConstituencies)

In [ ]:
# Example constraint: https://stackoverflow.com/questions/7844460/foreign-key-to-multiple-tables
CreatePollMeta = """Create Table PollMeta
    (
    PollID AS CONVERT(CHAR(8),PollDate,112) + Pollster + PollType + PollScope PERSISTED PRIMARY KEY,
    Pollster VARCHAR(100) FOREIGN KEY REFERENCES Pollsters(PollsterName) NOT NULL,
    PollType VARCHAR(25) FOREIGN KEY REFERENCES RegionTypes(RegionType) NOT NULL,
    PollDate DATE NOT NULL,
    PollScope VARCHAR(50) NOT NULL,
    PollScopeAll BIT NOT NULL,
    PollScopeRegion VARCHAR(75) FOREIGN KEY REFERENCES Regions(RegionID),
    PollScopeConst VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName),
    SampleSize INT,
    CONSTRAINT CheckPollScope CHECK(
        PollScopeAll +
        CASE WHEN PollScopeRegion IS NULL THEN 0 ELSE 1 END +
        CASE WHEN PollScopeConst IS NULL THEN 0 ELSE 1 END
        = 1)    
    )"""

In [ ]:
engine.execute(CreatePollMeta)

In [ ]:
CreatePollDetails = """Create Table PollDetails
    (
    PollDetailsID AS PollID + RegionID + Party PERSISTED PRIMARY KEY,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    RegionID VARCHAR(75) FOREIGN KEY REFERENCES Regions(RegionID) NOT NULL,
    Party VARCHAR(7) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [ ]:
engine.execute(CreatePollDetails)

In [ ]:
CreateCandidates = """Create Table Candidates
    (
    CandidateID AS Constituency + Party PERSISTED PRIMARY KEY,
    Constituency VARCHAR(50) FOREIGN KEY REFERENCES Constituencies(ConstituencyName) NOT NULL,
    Party VARCHAR(7) FOREIGN KEY REFERENCES Parties(PartyAbbreviation) NOT NULL,
    SittingMP BIT,
    FormerMP BIT,
    FirstName VARCHAR(25),
    Surname VARCHAR(25)
    )"""

In [ ]:
engine.execute(CreateCandidates)

In [ ]:
CreatePollAnalysisMeta = """Create Table PollAnalysisMeta
    (
    PollAnalysisID AS CONVERT(CHAR(8),PollAnalysisDate,112) + PollID + PollAnalysisAlgorithm PERSISTED PRIMARY KEY,
    PollAnalysisDate DATE NOT NULL,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    PollAnalysisAlgorithm VARCHAR(50) NOT NULL
    )"""

In [ ]:
engine.execute(CreatePollAnalysisMeta)

In [ ]:
CreatePollAnalysisRegions = """Create Table PollAnalysisRegions
    (
    PollAnalysisRegionID AS PollDetailsID + PollAnalysisID PERSISTED PRIMARY KEY,
    PollDetailsID VARCHAR(265) FOREIGN KEY REFERENCES PollDetails(PollDetailsID) NOT NULL,
    PollAnalysisID VARCHAR(241) FOREIGN KEY REFERENCES PollAnalysisMeta(PollAnalysisID) NOT NULL,
    Swing DECIMAL(9,8) NOT NULL
    )"""

In [ ]:
engine.execute(CreatePollAnalysisRegions)

In [ ]:
CreatePollAnalysisConstituencies = """Create Table PollAnalysisConstituencies
    (
    PollAnalysisRegionID VARCHAR(506) FOREIGN KEY REFERENCES PollAnalysisRegions(PollAnalysisRegionID) NOT NULL,
    CandidateID VARCHAR(57) FOREIGN KEY REFERENCES Candidates(CandidateID) NOT NULL,
    PollAnalysisConstituenciesID AS PollAnalysisRegionID + CandidateID PERSISTED PRIMARY KEY,    
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [ ]:
engine.execute(CreatePollAnalysisConstituencies)

In [ ]:
CreateElectionPredictionMeta = """Create Table ElectionPredictionMeta
    (
    ElectionPredictionID AS CONVERT(CHAR(8),AnalysisRunDate,112)  + CONVERT(CHAR(8),ElectionPredictionDate,112) + ElectionAlgorithm PERSISTED PRIMARY KEY,
    AnalysisRunDate Date NOT NULL,
    ElectionPredictionDate Date NOT NULL,
    ElectionAlgorithm VARCHAR(50) NOT NULL
    )"""

In [ ]:
engine.execute(CreateElectionPredictionMeta)

In [ ]:
CreateElectionPredictionPollsUsed = """Create Table ElectionPredictionPollsUsed
    (
    EPPUID AS PollID + ElectionPredictionID PERSISTED PRIMARY KEY,
    PollID VARCHAR(183) FOREIGN KEY REFERENCES PollMeta(PollID) NOT NULL,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL
    )"""

In [ ]:
engine.execute(CreateElectionPredictionPollsUsed)

In [ ]:
CreateElectionPredictionData = """Create Table ElectionPredictionData
    (
    ElectionPredictionDataID AS ElectionPredictionID + RegionID + Party PERSISTED PRIMARY KEY,
    ElectionPredictionID VARCHAR(66) FOREIGN KEY REFERENCES ElectionPredictionMeta(ElectionPredictionID) NOT NULL,
    RegionID VARCHAR(75) REFERENCES Regions(RegionID) NOT NULL,
    Party VARCHAR(7) REFERENCES Parties(PartyAbbreviation) NOT NULL,
    VoteShare DECIMAL(9,8) NOT NULL
    )"""

In [ ]:
engine.execute(CreateElectionPredictionData)

In [ ]:
#Get the table names currently in the database
insp = sqlalchemy.inspect(engine)
print(insp.get_table_names())

In [ ]:
conn.close()